# Scoping

Family midsize farm company from USA with experience in peas growing interested in expansion to the Canadian market according to the new government support program farmland leasing for 10 years in SK and MB.

Need to find out the most appropriate territories for peas growing for the mid-time period in this region according to rent options.

# ETL (Extract, Transform, Load)

In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sb
import folium
plt.rcParams['figure.figsize'] = [12, 8]

Import data sets connected to territories from open sources

## Yield

In [ ]:
path1 = r'D:\Education\Palette Agro\Analitics\Assignments\3\rm-yields-data.csv'
df_sk = pd.read_csv(path1)

# error with basic code reading 
# df_sk=pd.read_csv('D:\Education\Palette Agro\Analitics\Assignments\3\rm-yields-data.csv')

In [ ]:
path2 = r'D:\Education\Palette Agro\Analitics\Assignments\3\MMPP - Yield by Soil Type Browser.xlsx'
df_mb = pd.read_excel(path2)

# error with basic code reading
# df_mb=pd.read_excel('D:\Education\Palette Agro\Analitics\Assignments\3MMPP - Yield by Soil Type Browser.xlsx')

## Shapefiles

In [ ]:
path3 = r'D:\Education\Palette Agro\Analitics\Assignments\3\RuralMunicipality.shp'
gdf_sk = gpd.read_file(path3)

# error with basic code reading
# gdf_sk=gpd.read_file('D:\Education\Palette Agro\Analitics\Assignments\3\RuralMunicipality.shp')

In [ ]:
path4 = r'D:\Education\Palette Agro\Analitics\Assignments\3\MB_Municipal_Boundaries.shp'
gdf_mb = gpd.read_file(path4)

# error with basic code reading
#gdf_mb=gpd.read_file('D:\Education\Palette Agro\Analitics\Assignments\3MB_Municipal_Boundaries.shp')

## Transforming

Basic analysis of datasets for understanding of next manipulations

In [ ]:
# Info about columns from SK set
df_sk.info()

In [ ]:
# Info about columns from MB set
df_mb.info()

In [ ]:
# Table heads from SK set
df_sk.head()

In [ ]:
# Table heads from MB set
df_mb.head()

In [ ]:
# SK set main numeric info
df_sk.describe()

In [ ]:
# MB set main numeric info
df_mb.describe()

Datasets have different structures and measurements.
Have to be brought to a unified form.

In [ ]:
df_mb['Yield/acre(Metric)']=df_mb['Yield/acre(Metric)'].str.replace(' Tonnes', '') # Replacing Tonnes
df_mb['Yield/acre(Metric)']=df_mb['Yield/acre(Metric)'].replace('Tolerance', np.NaN) # Replacing Tolerance
df_mb['Yield/acre(Metric)']=df_mb['Yield/acre(Metric)'].astype(float) # Changing object to float data type

In [ ]:
df_mb_pivot=pd.pivot_table(df_mb.drop(columns=['Yield/acre(Metric).1', 'Yield/acre(Imperial)', 'Soil', 'Farms' ]),
               index=['Risk Area / R.M.', 'Year'], columns='Crop', values='Yield/acre(Metric)')\
               .reset_index()

In [ ]:
df_sk.columns

In [ ]:
df_mb_pivot.columns

In [ ]:
# Rename columns for the same in both datasets and drop some data that we are not going to use
df_mb_clean=df_mb_pivot.rename(columns={
    'Risk Area / R.M.': 'RM', 
    'ARGENTINE CANOLA': 'Canola', 
    'BARLEY':'Barley',
    'CANARYSEED': 'Canary Seed', 
    'DURUM WHEAT': 'Durum Wheat', 
     'LENTILS': 'Lentils', 
     'OATS': 'Oats',
    'RED SPRING WHEAT': 'Spring Wheat', 
    'WHITE PEA BEANS': 'Peas'}) \
        .drop(columns=['ALFALFA', 'FABABEANS', 'FABABEANS', 'POLISH CANOLA'] )

In [ ]:
df_sk_clean=df_sk.rename(columns={'Durum': 'Durum Wheat'}).drop(columns=['Winter Wheat', 'Mustard', 'Sunflowers', 'Fall Rye', 'Spring Rye', 'Tame Hay','Flax', 'Chickpeas' ] )

In [ ]:
df_sk_clean

In [ ]:
df_mb_clean

In [ ]:
# Crop conversion in MB tonnes to bushel
df_mb_clean['Canola']=df_mb_clean['Canola'] * 44.092
df_mb_clean['Barley']=df_mb_clean['Barley'] * 45.93
df_mb_clean['Canary Seed']=df_mb_clean['Canary Seed'] * 44.092
df_mb_clean['Durum Wheat']=df_mb_clean['Durum Wheat'] * 36.74
df_mb_clean['Lentils']=df_mb_clean['Lentils'] * 36.74
df_mb_clean['Oats']=df_mb_clean['Oats'] * 64.842
df_mb_clean['Spring Wheat']=df_mb_clean['Spring Wheat'] * 36.74
df_mb_clean['Peas']=df_mb_clean['Peas'] *  36.74


In [ ]:
# Crop Conversion in SK - pounds to bushels
df_sk_clean['Lentils']=df_sk_clean['Lentils'] / 60
df_sk_clean['Canary Seed']=df_sk_clean['Canary Seed'] / 50

In [ ]:
# Creating province column
df_mb_clean['Province'] = 'MB'
df_sk_clean['Province'] = 'SK'

In [ ]:
# Merging of data sets
df = pd.concat([df_mb_clean, df_sk_clean])

# EDA (Exploratory Data Analysis)

## Missing Values

Analyzing our dataset for missing data for columns with Crops

In [ ]:
exclude_columns = ['RM', 'Year', 'Province']
selected_columns = [col for col in df.columns if col not in exclude_columns]
df_selected = df[selected_columns]

colors = ['dodgerblue' if col == 'Oats' else 'lightgray' for col in selected_columns]  #some issue with column coloring

missing_values = df_selected.isna().sum().sort_values()
missing_values.plot(kind='bar', color=colors)
plt.title('Missing Values - Full data from 1938 to 2022', color='Black')
plt.xlabel('Crops')
plt.ylabel('Count of Missing values')
plt.axhline(len(df_selected) / 2, linestyle='--', color='red')
plt.axhline(len(df_selected) / 4, linestyle='--', color='yellow')
plt.axhline(len(df_selected) / 10, linestyle='--', color='green')
plt.show()

We focused on short/mid-period so checking only last 10 years

In [ ]:
exclude_columns = ['RM', 'Year', 'Province']
filtered_df = df[df['Year'] > 2012]
selected_columns = [col for col in filtered_df.columns if col not in exclude_columns]
df_selected = filtered_df[selected_columns]

colors = ['dodgerblue' if col == 'Lentils' else 'lightgray' for col in selected_columns] #some issue with column coloring

missing_values = df_selected.isna().sum().sort_values()
missing_values.plot(kind='bar', color=colors)
plt.title('Missing Values - Data from 2013 to 2022', color='Black')
plt.xlabel('Crops')
plt.ylabel('Count of Missing values')
plt.axhline(len(df_selected)/2, linestyle='--', color='red')
plt.axhline(len(df_selected)/4, linestyle='--', color='yellow')
plt.axhline(len(df_selected)/10, linestyle='--', color='green')
plt.show()

## Outliers

In [ ]:
crop_columns=['Peas']

## Histograms

In [ ]:
df[crop_columns].hist(bins=200)
plt.show()

## Boxplots

In [ ]:
df[crop_columns].boxplot()

Find extreme points and their number

In [ ]:
df.loc[df['Peas']>75]

## GIS Analyis

In [ ]:
# CRS --> Cordinate Reference Systems
gdf_mb['geometry'].crs

In [ ]:
# Standardazing CRS formats
gdf_sk['geometry']=gdf_sk['geometry'].to_crs('epsg:4326')
gdf_mb['geometry']=gdf_mb['geometry'].to_crs('epsg:4326')

In [ ]:
# Renaming column name
gdf_sk['RMNO']=gdf_sk['RMNO'].astype(int)

In [ ]:
#set(gdf['RM'].unique()) - set(df_sk_clean['RM'].unique())

In [ ]:
gdf_mb['MUNI_NAME']=gdf_mb['MUNI_NAME'].str.replace('RM OF ','')

In [ ]:
gdf=pd.concat([gdf_sk[['RMNO','geometry']].rename(columns={'RMNO':'RM'}),gdf_mb[['MUNI_NAME', 'geometry']].rename(columns={'MUNI_NAME':'RM'})])

In [ ]:
# gdf.plot()not interactive
# gdf.explore()

In [ ]:
gdf['RM']=gdf['RM'].astype('string')
df['RM']=df['RM'].astype('string')

In [ ]:
# SQL inner join
df_gdf=pd.merge(gdf, df, on='RM', how='inner') 

In [ ]:
print('Before merging', gdf['RM'].nunique())
print('After merging',df_gdf['RM'].nunique())

In [ ]:
ds=df_gdf.copy()

### GIS Visualization

Explore function

In [ ]:
m=ds.loc[ds['Year']==2021].explore(column='Peas', 
                                   legend=True, cmap='Greens',tooltip= ['Peas', 'RM'], tiles='Stamen Toner') 
# Plot() is good for showing up in GitHub, Explore() is good for interactive map and saving as HTML

# Adding a title with dark orange color to the folium map
title_html = '''
                 <h3 align="center" style="font-size:30px; color:Green;"><b> Canola Yield in 2021 </b></h3>
             '''
m.get_root().html.add_child(folium.Element(title_html))
m.save(r'D:\Education\Palette Agro\Analitics\Assignments\3\Peas_2021.html')

#### Plot

First glance at the territories for the lastest data period

In [ ]:
ds.loc[ds['Year']==2022].plot(column='Peas', legend=True, cmap='Blues')

Visual differences for the last 10 years

In [ ]:
years = list(range(2014, 2023))

num_years = len(years)
num_cols = 3
num_rows = (num_years + num_cols - 1) // num_cols

fig, axes = plt.subplots(num_rows, num_cols, figsize=(12, 8))
axes = axes.flatten()

for i, year in enumerate(years):
    if i < num_years:
        ax = axes[i]
        ds.loc[ds['Year'] == year].plot(column='Peas', legend=False, cmap='Blues', ax=ax)
        ax.set_title(f'Year {year}')
        ax.set_axis_off()
    else:
        fig.delaxes(axes[i])

plt.tight_layout()
plt.show()

## Aggregations

More detailed and visible differences

In [ ]:
df_13_22=df.loc[df['Year']>2013].sort_values(['RM', 'Year'])

In [ ]:
fig, axes = plt.subplots(3, 3, figsize=(12, 8))
years = df_13_22['Year'].unique()

for i, year in enumerate(years):
    ax = axes[i//3, i%3]
    merged_df = pd.merge(gdf, df_13_22.loc[df_13_22['Year'] == year], on='RM')
    merged_df.plot(column='Peas', cmap='Blues', legend=True, ax=ax)
    ax.set_title(f'Peas Yield in {year}', color='Black', size=12)

    ax.set_axis_off()

plt.tight_layout()
plt.show()

Comparison with mean for 10 and 5 last years for positive trending on territories

In [ ]:
mean_peas_1 = df_13_22[(df_13_22['Year'] >= 2013) & (df_13_22['Year'] <= 2022)].groupby('RM')['Peas'].mean()
mean_peas_2 = df_13_22[(df_13_22['Year'] >= 2017) & (df_13_22['Year'] <= 2022)].groupby('RM')['Peas'].mean()

new_df_mean_years = df_13_22[['RM', 'Year', 'Province', 'Peas']]

new_df_mean_years['Mean_Peas_1'] = df_13_22['RM'].map(mean_peas_1)
new_df_mean_years['Mean_Peas_2'] = df_13_22['RM'].map(mean_peas_2)
new_df_mean_years['Mean_Peas_Difference'] = new_df_mean_years['Mean_Peas_2'] - new_df_mean_years['Mean_Peas_1']

In [ ]:
positive_rm = new_df_mean_years[new_df_mean_years['Mean_Peas_Difference'] > 0].drop_duplicates(subset='RM')[['RM']]

In [ ]:
agg_funcs = {
    column: ['mean', 'std', 'median'] for column in crop_columns
}

df_agg = new_df_mean_years.set_index('RM')[crop_columns].groupby('RM').agg(agg_funcs)
df_agg.columns = ['_'.join(col).strip() for col in df_agg.columns.values]

df_agg=df_agg.dropna(subset='Peas_mean')

Dataframe of RM with positive trend of Yield

In [ ]:
positive_rm

# Feature Selection

## Filtered Methods

In [ ]:
# ANOVA
# Pearson Correlation

## Wrapper

In [ ]:
## Recurisive Feature Elimnation
## Backward feature elimination

## Emedded 

In [ ]:
# Decision Tree
# Lasso Reg

# Standardizing/Split

In [ ]:
# Only for Supervised ML, not unsupervised

# Training Models

In [ ]:
# Use default parameters - not advised

# Parameter tuning
# Use always grid search

## K-Means Clustering

In [ ]:
# Importing library
from sklearn.cluster import KMeans 

df_agg_peas= df_agg[['Peas_mean', 'Peas_std']]

# Let's define our features
X = df_agg_peas.copy()

from sklearn.metrics import silhouette_score
n_clusters = [2,3,4,5,6,7,8,9,10,11,12,13,14,15] # number of clusters
clusters_inertia = [] # inertia of clusters
s_scores = [] # silhouette scores

for n in n_clusters:
    KM = KMeans(n_clusters=n, init='k-means++', n_init=10).fit(X)
    clusters_inertia.append(KM.inertia_)    # data for the elbow method
    silhouette_avg = silhouette_score(X, KM.labels_)
    s_scores.append(silhouette_avg) # data for the silhouette score method

## Elbow Metod

In [ ]:
fig, ax = plt.subplots(figsize=(12,5))
ax.plot(n_clusters, clusters_inertia, 'o-', color='dodgerblue', label='Elbow Method')
ax.set_title("Elbow Method for Peas data")
ax.set_xlabel("Number of Clusters")
ax.set_ylabel("Clusters Inertia")
ax.axvline(4, ls="--", c="lightcoral")
ax.axvline(5, ls="--", c="lightcoral")
ax.axvline(6, ls="--", c="lightcoral")
plt.grid()
plt.legend()
plt.show()

## Silhouette Score

In [ ]:
fig, ax = plt.subplots(figsize=(12,5))
ax.plot(n_clusters, s_scores, 's-', color='dodgerblue', label='Silhouette Score Method')
ax.set_title("Silhouette Score Method for Peas data")
ax.set_xlabel("Number of Clusters")
ax.set_ylabel("Silhouette Score")
ax.axvline(6, ls="--", c="lightcoral")
plt.grid()
plt.legend()
plt.show()


In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=5, init='k-means++', random_state=42, n_init=10)
df_agg_peas['Clusters_5']=kmeans.fit_predict(df_agg_peas)

In [ ]:
kmeans = KMeans(n_clusters=7, init='k-means++', random_state=42)
df_agg_peas['Clusters_7']=kmeans.fit_predict(df_agg_peas)

In [ ]:
sb.scatterplot(data=df_agg_peas, x='Peas_mean', y='Peas_std', hue='Clusters_5')
plt.title('Peas Clustering Mean and Std | 2013-2022 | K-Means ', color='Black', size =12)
plt.show()

In [ ]:
sb.scatterplot(data=df_agg_peas, x='Peas_mean', y='Peas_std', hue='Clusters_7')
plt.title('Peas Clustering Mean and Std | 2013-2022 | K-Means ', color='black', size =12)
plt.show()

In [ ]:
df_agg_peas

In [ ]:
pd.merge(
    gdf,
    df_agg_peas,
    on='RM'
).explore(column='Clusters_7', legend='True', k=7, scheme='naturalbreaks', cmap='Blues')

## Ranking clusters based on Mean

In [ ]:
df_agg_peas.groupby('Clusters_7').mean()\
    .sort_values('Peas_mean')[['Peas_mean',	'Peas_std']]

In [ ]:
# Ranking based on the mean / Manual repositioning
df_agg_peas['Clusters_7_ranked']=df_agg_peas['Clusters_7'].replace(to_replace={
    3:1,
    6:3,
    0:4,
    5:2,
    2:0,
    4:5,
    1:6
})

In [ ]:
df_agg_peas.groupby('Clusters_7').mean()\
    .sort_values('Peas_std')[['Peas_mean',	'Peas_std']]

In [ ]:
# Ranking based on the STD or volatility / Manual repositioning
df_agg_peas['Clusters_7_ranked_std']=df_agg_peas['Clusters_7'].replace(to_replace={
    2:0,
    5:1,
    4:2,
    1:3,
    3:4,
    0:5,
    6:6
})

In [ ]:
df_agg_peas.head()

Yield of territories by mean

In [ ]:
pd.merge(
    gdf,
    df_agg_peas,
    on='RM'
).explore(column='Clusters_7_ranked', legend='True', k=7, scheme='naturalbreaks', cmap='Blues')

Yield of territories by STD

In [ ]:
pd.merge(
    gdf,
    df_agg_peas,
    on='RM'
).explore(column='Clusters_7_ranked_std', legend='True', k=7, scheme='naturalbreaks', cmap='Blues')

The same data but with default plot

In [ ]:
merged_gdf = pd.merge(gdf, df_agg_peas, on='RM')

merged_gdf.plot(column='Clusters_7_ranked_std', legend=False, k=7, scheme='naturalbreaks', cmap='Blues')
plt.show()

For more focus going to filter by positive trend

In [ ]:
merged_df = positive_rm.merge(df_agg_peas, on='RM')

merged_gdf = gdf.merge(merged_df, on='RM')

fig, ax = plt.subplots(1, 1, figsize=(12, 10))
merged_gdf.plot(column='Clusters_7_ranked_std', legend=False, k=7, scheme='naturalbreaks', cmap='Blues', ax=ax)

ax.set_title('Positive trend for the last 5 years')
plt.show()

Creating a map with the best Yield and positive trend territories

In [ ]:
merged_df = positive_rm.merge(df_agg_peas[df_agg_peas['Clusters_7_ranked_std'].isin([0])], on='RM', how='inner')
merged_gdf = gdf.merge(merged_df, on='RM')
rm_values = merged_gdf['RM'].tolist()

fig, ax = plt.subplots(1, 1, figsize=(12, 10))
merged_gdf.plot(column='Clusters_7_ranked_std', legend=False, k=7, scheme='naturalbreaks', cmap='Blues_r', ax=ax)

ax.set_title('Territories for primary focus')
plt.show()

Making a list of territories for short/mid-period renting for growing Peas

In [ ]:
rm_int_values = [int(value) for value in rm_values]

print(rm_int_values)

In [ ]:
mean_by_cluster = df_agg_peas.groupby('Clusters_7_ranked_std').mean()
print(mean_by_cluster)

# Error  (Supervised ML)

In [ ]:
# MAE (Mean Absolute Error)
# RMSE(Mean Squared Error)

# Based on above errors, find the min error model.
# Check error distribution
# Look at the difference
# Make a scatterplot

# Deployment

In [ ]:
# ML engineers deploys models with SD

# AWS Sagemaker

# Monitoring

In [ ]:
# Look at error

# If something wrong go back to step 1